In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv
import numpy as np
import re

#### Достанем список всех страниц из папки

In [2]:
pages = os.listdir(os.getcwd() + '/pages')

#### Напишем функцию записи нужных данных в словарь

In [171]:
def get_page_data(files):
    lst = list()
    for file in files:
        data = dict()
        with open(os.getcwd() + '/pages/' + file, 'r', encoding='utf-8') as f: 
            soup = BeautifulSoup(f, 'lxml')

            try:
                data['Название'] = soup.find('h1', class_=re.compile('\.*--title--\.*')).text
            except:
                data['Название'] = np.nan
            try:
                data['Адрес'] = soup.find('address', class_=re.compile('\.*--address--\.*')).text.replace('На карте', '')
            except:
                data['Адрес'] = np.nan
            try:    
                item = soup.find_all('li', class_=re.compile('\.*--underground--\.*'))
                for i,div in enumerate(item):
                    value = div.find('span', class_=re.compile('\.*--underground_time--\.*')).text
                    data[f'Метро {i}'] = value
            except:
                pass
            try:
                data['Цена'] = soup.find('span', class_=re.compile('\.*--price_value--\.*')).text[:-1].replace('\xa0', '')
            except:
                data['Цена'] = np.nan
            try:
                data['Цена за квадрат'] = soup.find('div', class_=re.compile('\.*--price_per_meter--\.*')).text[:-5].replace('\xa0', '')
            except:
                data['Цена за квадрат'] = np.nan
            try:    
                info = soup.find_all('div', class_=re.compile('\.*--info--\.*'))
                for div in info:
                    value = div.find('div', class_=re.compile('\.*--info-value--\.*')).text
                    name = div.find('div', class_=re.compile('\.*--info-title--\.*')).text
                    data[name] = value
            except:
                pass
            try:    
                item = soup.find_all('div', class_=re.compile('\.*--item--\.*'))
                for div in item:
                    value = div.find('div', class_=re.compile('\.*--value--\.*')).text
                    name = div.find('div', class_=re.compile('\.*--name--\.*')).text
                    data[name] = value
            except:
                pass
            lst.append(data)
    df = pd.DataFrame(lst)
    df.to_csv('cian.csv', index=False)
    return df

In [172]:
get_page_data(pages).head()

,Аварийность,Адрес,Безопасность,Газоснабжение,Год постройки,Жилая,Инфраструктура,Кухня,Лифты,Метро 0,...,Построен,Срок сдачи,Строительная серия,Тип дома,Тип перекрытий,Транспорт,Цена,Цена за квадрат,Экология,Этаж
0,Нет,"Московская область, Мытищи, мкр. Ярославский, ...",NaN,NaN,2012,24 м²,NaN,10 м²,"2 пассажирских, 1 грузовой",⋅ 15 мин. на транспорте,...,2012,NaN,КОПЭ Парус,Панельный,Железобетонные,NaN,6550000,136 458,NaN,12 из 25
1,Нет,"Москва, ЗАО, р-н Дорогомилово, Кутузовский про...",NaN,Центральное,1957,45 м²,NaN,NaN,Нет,⋅ 13 мин. пешком,...,1934,NaN,NaN,Кирпичный,Железобетонные,NaN,26000000,406 250,NaN,4 из 7
2,Нет,"Москва, ЦАО, р-н Тверской, 1-й Колобовский пер...",NaN,NaN,1917,"23,4 м²",NaN,"8,9 м²",Нет,⋅ 5 мин. пешком,...,1917,NaN,NaN,Кирпичный,Деревянные,NaN,18000000,450 000,NaN,2 из 3
3,Нет,"Москва, ЗАО, р-н Раменки, Мосфильмовская ул., 8",NaN,NaN,2011,63 м²,NaN,20 м²,Нет,⋅ 11 мин. на транспорте,...,NaN,NaN,NaN,Монолитный,Железобетонные,NaN,59900000,611 224,NaN,33 из 54
4,Нет,"Москва, ЦАО, р-н Хамовники, Комсомольский прос...",NaN,NaN,2014,48 м²,NaN,NaN,1 пассажирский,⋅ 10 мин. пешком,...,NaN,NaN,NaN,Монолитный,NaN,NaN,37900000,563 150,NaN,4 из 5
